In [5]:
def add_():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))
10


In [6]:
# 混合式编程
import torch
from torch import nn
from d2l import torch as d2l

# 生产网络的工厂模式
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
                        nn.ReLU(),
                        nn.Linear(256, 128),
                        nn.ReLU(),
                        nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[-0.1692, -0.0002]], grad_fn=<AddmmBackward0>)

In [7]:
# 通过使用torch.jit.script(net)函数来转换模型，我们就有能力编译和优化多层感知机中的计算，而模型的计算结果保持不变
net = torch.jit.script(net)
net(x)

tensor([[-0.1692, -0.0002]], grad_fn=<AddmmBackward0>)

In [8]:
# 我们编写与之前相同的代码，再使用torch.jit.script简单地转换模型，当完成这些任务后，网络就将得到优化（我们将在下面对性能进行基准测试）
# 通过混合式编程加速
# 为了证明通过编译获得了性能改进，我们比较了混合变化层前后执行net(x)所需的时间。让我们先定义一个度量时间的类

In [9]:
class Benchmark:
    """用于测量运行时间"""
    def __init__(self, description='Done'):
        self.description=description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

In [16]:
net = get_net()
with Benchmark('无torchscript'):
    for i in range(1000):
        net(x)

net = torch.jit.script(net)
with Benchmark('有torchscript'):
    for i in range(1000):
        net(x)

无torchscript: 0.0299 sec
有torchscript: 0.0233 sec


In [17]:
# 序列化
# 编译模型的好处之一是我们可以将模型及其参数序列化（保存到）磁盘。这允许这些训练好的模型不熟到其他设备上，并且还能方便地使用其他前端编程语言。同时，通常编译模型的代码执行速度也比命令式编程更快，让我们看看save的实际功能。

In [18]:
net.save('my_mlp')

In [19]:
!ls -lh my_mlp*

-rw-rw-r-- 1 cookie cookie 652K May 24 21:21 my_mlp
